In [1]:
import os
from glob import glob
from tqdm import tqdm

import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import albumentations as A
from albumentations.pytorch import ToTensorV2

import sys
sys.path.append('/root/dacon-epitope/dacon-sem')
from src.datasets.sem_dataset import SEMDataset
from src.datasets.sem_datamodule import SEMDataModule

In [2]:
data_path = os.path.abspath('/shared/Samsung/')
simulation_sem_paths = os.path.join(data_path, 'simulation_data', 'SEM', '*', '*', '*.png')
simulation_sem_paths = np.array(sorted(glob(simulation_sem_paths)))
simulation_depth_paths = os.path.join(data_path, 'simulation_data', 'Depth', '*', '*', '*.png')
simulation_depth_paths = np.array(sorted(glob(simulation_depth_paths) + glob(simulation_depth_paths)))
simulation_data_len = len(simulation_sem_paths)

train_sem_paths = os.path.join(data_path, 'train', 'SEM', '*', '*', '*.png')
train_sem_paths = np.array(sorted(glob(train_sem_paths)))
train_data_len = len(train_sem_paths)

test_sem_paths = os.path.join(data_path, 'test', 'SEM', '*.png')
test_sem_paths = np.array(sorted(glob(test_sem_paths)))
test_data_len = len(test_sem_paths)

result_path = os.path.abspath('/shared/youngkim/dacon-sem/submissions/')

In [3]:
def draw_line_by_row(paths, save_dir_name, file_name='heatmap.png'):

    from matplotlib import cm
    from matplotlib.colors import ListedColormap, BoundaryNorm

    if(type(paths) is str):
        paths = sorted(glob(paths))
    
    pixel_tables=np.zeros((72,48,len(paths)),dtype=np.uint8)
    
    for i in range(0,len(paths)):
        img = cv2.imread(paths[i], cv2.IMREAD_GRAYSCALE)
        pixel_tables[:,:,i] = img.copy()
    
    
    heat_map = np.zeros((256,48,72))
    for i in range(0,pixel_tables.shape[0]):
        for j in range(0,pixel_tables.shape[1]):
            t_unique, t_counts = np.unique(pixel_tables[i,j,:],return_counts=True)
            for t_height,t_value in zip(t_unique,t_counts):
                heat_map[t_height,j,i]=t_value
    
    bounds = np.append(1,np.arange(1000,30001,2000))
    jet = cm.get_cmap('jet', 256)
    newcolors = jet(np.linspace(0, 1, 256))
    black = np.array([0/256, 0/256, 0/256, 1])
    #newcolors[:1, :] = black #colorbar customize
    newcmp = ListedColormap(newcolors)
    newcmp.set_over(newcolors[-1])
    newcmp.set_under(black)
    norm = BoundaryNorm(bounds, newcmp.N)
    
    fig=plt.figure(figsize=(34,20),constrained_layout=True,)
    
    subplot = fig.subplots(nrows=3, ncols=24)
    
    #plt.subplots_adjust(left=0.05,right=1,top=1,bottom=0.2,wspace=0.3,hspace=0.1)
    
    i=0
    for ax in subplot.flat:
        im_heatmap = ax.pcolor(heat_map[:,:,i],cmap=newcmp,#edgecolors='black',
                               vmin=0,vmax=heat_map.max(),norm=norm);
        i=i+1;
    
    plt.tight_layout()
    plt.subplots_adjust(bottom=0.1)
    cb_ax=fig.add_axes([0.25,0.025,0.5,0.05])  ##<= (left,bottom,width,height)
    plt.colorbar(im_heatmap,cax=cb_ax,orientation='horizontal',pad=0.02,ticks=bounds,extend='both',norm=norm)
    
    os.makedirs(save_dir_name, exist_ok=True)
    heat_img_file_path = os.path.join(save_dir_name,file_name)
    plt.savefig(heat_img_file_path,dpi=fig.dpi)
    
    plt.cla()
    plt.clf()
    plt.close()


In [ ]:
depth_path_all = '/shared/Samsung/simulation_data/Depth/*/*/*.png'
simulation_depth_paths_all = sorted(glob(depth_path_all))
save_dir = os.path.abspath('/root/dacon-epitope/dacon-sem/notebooks')

draw_line_by_row(np.unique(simulation_depth_paths_all), save_dir)


In [4]:
depth_path_all = os.path.abspath('/shared/youngkim/dacon-sem/submissions/*')
simulation_depth_paths_all = sorted(glob(depth_path_all))
save_dir = os.path.abspath('/root/dacon-epitope/dacon-sem/notebooks')

draw_line_by_row(np.unique(simulation_depth_paths_all), save_dir, 'heatmap_pred.png')

/tmp/ipykernel_352126/1004557400.py:41: MatplotlibDeprecationWarning: Passing parameters norm and vmin/vmax simultaneously is deprecated since 3.3 and will become an error two minor releases later. Please pass vmin/vmax directly to the norm when creating it.
  im_heatmap = ax.pcolor(heat_map[:,:,i],cmap=newcmp,#edgecolors='black',
/tmp/ipykernel_352126/1004557400.py:45: UserWarning: This figure was using constrained_layout==True, but that is incompatible with subplots_adjust and or tight_layout: setting constrained_layout==False. 
  plt.tight_layout()
/tmp/ipykernel_352126/1004557400.py:48: MatplotlibDeprecationWarning: The 'norm' parameter to Colorbar has no effect because it is overridden by the mappable; it is deprecated since 3.3 and will be removed two minor releases later.
  plt.colorbar(im_heatmap,cax=cb_ax,orientation='horizontal',pad=0.02,ticks=bounds,extend='both',norm=norm)
